# Keras의 Sequential 모델을 사용한 틴더의 relationship 성공 여부 예측 

<span style="color: blue; font-size: 18pt">
0. pandas 라이브러리 임포트하기
</span>

In [1]:
import pandas as pd

<span style="color: blue; font-size: 18pt">
1. 데이터셋을 pandas로 읽어오기
</span>

In [2]:
df_train = pd.read_csv("C:/Users/user/Downloads/final/5_Refined Dataset_(1).csv")


- 이번 딥러닝 프로젝트에 사용하는 데이터셋은 <span style="color: red">"5_Refined Dataset_(1)"</span> <span style="color: red">"5_Refined Dataset_(2)"</span>로 이루어져 있습니다. 

- 모델을 학습시킬 때는 <span style="color: red">"5_Refined Dataset_(1)"</span>에서 7:3을 나눠 학습 후 정확도를 측정합니다.

- y값이 존재하지 않는 <span style="color: red">"5_Refined Dataset_(2)"</span> 데이터셋은 마지막에 완성된 모델을 통한 예측 시에 사용합니다.


In [3]:
df_train

,ID,Answer,Count,Percentage,It became a relationship,Segment type_gender,Segment type_mobile,Segment type_university,Segment type_web,"Segment Description_Meridian, Idaho",Segment Description_SegDiscrip_Others
0,292890.8970,0,0.000000,0.000000,0,0.0,0.0,0.0,1.0,1.0,0.0
1,292887.9870,0,0.000000,0.000000,0,0.0,0.0,0.0,1.0,1.0,0.0
2,292894.0656,0,499.173606,0.225255,0,1.0,0.0,0.0,0.0,1.0,0.0
3,292887.1180,0,0.000000,0.000000,0,0.0,0.0,0.0,1.0,1.0,0.0
4,292893.6561,0,455.925963,0.211360,0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1891,292887.5496,0,0.000000,0.000000,0,0.0,0.0,0.0,1.0,1.0,0.0
1892,292881.6932,0,1203.190399,0.312360,0,0.0,1.0,0.0,0.0,1.0,0.0
1893,292900.8499,0,806.378820,0.488025,0,1.0,0.0,0.0,0.0,1.0,0.0
1894,292893.8600,0,1149.529381,0.488984,0,1.0,0.0,0.0,0.0,1.0,0.0


<span style="color: blue; font-size: 18pt">
2. 딥러닝에 필요한 라이브러리 임포트하기
</span>

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split 

import numpy as np
import tensorflow as tf

In [5]:
# 실행할 때마다 같은 결과를 출력하기 위해 시드 설정
seed=32
np.random.seed(seed) 
tf.random.set_seed(seed) 

In [6]:
# 데이터를 np 기반으로 읽기 
dataset = np.loadtxt("C:/Users/user/Downloads/final/5_Refined Dataset_(1).csv", delimiter=",", skiprows=1)
X = dataset[:, [0, 1, 2, 3, 5, 6, 7, 8, 9, 10]]
Y = dataset[:, 4]

In [7]:
X

array([[2.92890897e+05, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.92887987e+05, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.92894066e+05, 0.00000000e+00, 4.99173606e+02, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [2.92900850e+05, 0.00000000e+00, 8.06378820e+02, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.92893860e+05, 0.00000000e+00, 1.14952938e+03, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.92890294e+05, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [8]:
Y

array([0., 0., 0., ..., 0., 0., 0.])

<span style="color: blue; font-size: 18pt">
3. 읽어온 데이터셋을 학습용/테스트용으로 분리하기
</span>

In [9]:
#학습 데이터 70 %, 테스트 데이터셋 30% 로 설정하기
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

In [10]:
print( "학습 데이터 개수 : %d  ,  테스트 데이터 개수 %d  "  %(len(X_train), len(X_test)))

학습 데이터 개수 : 1327  ,  테스트 데이터 개수 569  


In [11]:
print('● type(X_train) : ', type(X_train)) # train_test_split( ) 함수의 리턴 값은 numpy.ndarray 타입
print('● np.shape(X_train) : ',  np.shape(X_train)) # 학습 데이터 모양 
print('● len(X_train) : ', len(X_train), '개') # 학습 데이터 개수 
print(X_train)  #학습 데이터 속성 확인
print('-'*100)
print('● type(X_test) : ', type(X_test)) # train_test_split( ) 함수의 리턴 값은 numpy.ndarray 타입
print('● np.shape(X_test) : ',  np.shape(X_test)) # 테스트 데이터 모양 
print('● len(X_test) : ', len(X_test), '개') # 테스트 데이터 개수 
print(X_test)  #테스트 데이터 속성 확인

● type(X_train) :  <class 'numpy.ndarray'>
● np.shape(X_train) :  (1327, 10)
● len(X_train) :  1327 개
[[2.92899992e+05 0.00000000e+00 8.16087315e+02 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [2.92883475e+05 0.00000000e+00 9.78316067e+02 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [2.93549000e+05 0.00000000e+00 2.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 ...
 [2.93371000e+05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [2.92890037e+05 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [2.92892929e+05 0.00000000e+00 9.49847063e+02 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]]
----------------------------------------------------------------------------------------------------
● type(X_test) :  <class 'numpy.ndarray'>
● np.shape(X_test) :  (569, 10)
● len(X_test) :  569 개
[[2.92889540e+05 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  1.00000000e+00 0.00000000e+00]
 

<span style="color: blue; font-size: 18pt">
4. 딥러닝 모델 설계하기 
</span>

In [12]:
model = Sequential() #keras의 Sequential 모델 사용

#1번째 층 : 입력 x는 10개, 출력은 12개, 활성화함수 relu 
model.add(Dense(12, input_dim=10, activation='relu'))

#2번째 층 : 입력 x는 12개, 출력은 10개, 활성화함수 relu 
model.add(Dense(8, activation='relu'))

#3번째 층 : 입력 x는 10개, 출력은 1개, 활성화함수 sigmoid
model.add(Dense(1, activation='sigmoid'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                132       
                                                                 
 dense_1 (Dense)             (None, 8)                 104       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 245
Trainable params: 245
Non-trainable params: 0
_________________________________________________________________


<span style="color: blue; font-size: 18pt">
5. 딥러닝 모델 컴파일 및 실행하기 
</span>

In [14]:
# y값은 0/1을 예측하는 이진 분류이므로 loss='binary_crossentropy' 사용
model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy'])

<span style="color: orange; font-size: 12pt">
keras의 Sequential()에서 제공하는 fit( ) 기능으로 x, y, 학습 반복횟수, 배치 사이즈 설정
</span>

In [15]:
model.fit(X_train, Y_train, epochs=200, batch_size=10)

Epoch 1/200
133/133 [==============================] - 1s 1ms/step - loss: 481.1823 - accuracy: 0.5524
Epoch 2/200
133/133 [==============================] - 0s 1ms/step - loss: 64.0483 - accuracy: 0.5531
Epoch 3/200
133/133 [==============================] - 0s 1ms/step - loss: 77.0976 - accuracy: 0.5705
Epoch 4/200
133/133 [==============================] - 0s 1ms/step - loss: 77.5459 - accuracy: 0.5509
Epoch 5/200
133/133 [==============================] - 0s 1ms/step - loss: 41.8784 - accuracy: 0.5772
Epoch 6/200
133/133 [==============================] - 0s 1ms/step - loss: 62.1172 - accuracy: 0.5607
Epoch 7/200
133/133 [==============================] - 0s 1ms/step - loss: 62.8075 - accuracy: 0.5765
Epoch 8/200
133/133 [==============================] - 0s 1ms/step - loss: 86.6874 - accuracy: 0.5516
Epoch 9/200
133/133 [==============================] - 0s 1ms/step - loss: 59.7898 - accuracy: 0.5659
Epoch 10/200
133/133 [==============================] - 0s 1ms/step - loss: 101.2

133/133 [==============================] - 0s 2ms/step - loss: 65.2781 - accuracy: 0.5697
Epoch 81/200
133/133 [==============================] - 0s 2ms/step - loss: 53.8383 - accuracy: 0.5780
Epoch 82/200
133/133 [==============================] - 0s 2ms/step - loss: 49.3309 - accuracy: 0.5720
Epoch 83/200
133/133 [==============================] - 0s 2ms/step - loss: 70.7235 - accuracy: 0.5674
Epoch 84/200
133/133 [==============================] - 0s 2ms/step - loss: 44.1819 - accuracy: 0.5667
Epoch 85/200
133/133 [==============================] - 0s 2ms/step - loss: 46.8031 - accuracy: 0.5795
Epoch 86/200
133/133 [==============================] - 0s 2ms/step - loss: 68.2687 - accuracy: 0.5561
Epoch 87/200
133/133 [==============================] - 0s 2ms/step - loss: 42.1216 - accuracy: 0.5599
Epoch 88/200
133/133 [==============================] - 0s 2ms/step - loss: 48.0859 - accuracy: 0.5584
Epoch 89/200
133/133 [==============================] - 0s 2ms/step - loss: 59.6241 - 

133/133 [==============================] - 0s 2ms/step - loss: 79.3857 - accuracy: 0.5448
Epoch 160/200
133/133 [==============================] - 0s 2ms/step - loss: 38.4031 - accuracy: 0.5998
Epoch 161/200
133/133 [==============================] - 0s 2ms/step - loss: 59.2316 - accuracy: 0.5584
Epoch 162/200
133/133 [==============================] - 0s 2ms/step - loss: 44.1124 - accuracy: 0.5825
Epoch 163/200
133/133 [==============================] - 0s 2ms/step - loss: 38.6192 - accuracy: 0.5810
Epoch 164/200
133/133 [==============================] - 0s 2ms/step - loss: 38.8092 - accuracy: 0.5690
Epoch 165/200
133/133 [==============================] - 0s 2ms/step - loss: 51.5777 - accuracy: 0.5592
Epoch 166/200
133/133 [==============================] - 0s 2ms/step - loss: 38.7535 - accuracy: 0.5938
Epoch 167/200
133/133 [==============================] - 0s 2ms/step - loss: 46.2010 - accuracy: 0.5667
Epoch 168/200
133/133 [==============================] - 0s 2ms/step - loss: 4

<span style="color: blue; font-size: 18pt">
6. 테스트 데이터를 기반으로 딥러닝 성능 평가하기 
</span>

In [16]:
# model.evaluate() 사용하여 성능 평가 
print('● 테스트 데이터 개수 : ', len(X_test), '개')
ev = model.evaluate(X_test, Y_test, batch_size=1)  
print('● [ loss 오차 , accuracy 정확도] =' , ev) 

● 테스트 데이터 개수 :  569 개
569/569 [==============================] - 1s 2ms/step - loss: 17.7656 - accuracy: 0.6151
● [ loss 오차 , accuracy 정확도] = [17.765623092651367, 0.6151142120361328]


In [17]:
print("\n Test Accuracy : %.4f" % (model.evaluate(X_test, Y_test, batch_size=10)[1])) 

57/57 [==============================] - 0s 2ms/step - loss: 17.7655 - accuracy: 0.6151

 Test Accuracy : 0.6151


<span style="color: blue; font-size: 18pt">
7. 새로운 데이터를 대상으로 딥러닝 모델을 사용하여 예측하기 
</span>

In [18]:
# keras의 Sequential()에서 제공하는 predict() 기능 사용

test_data = pd.read_csv('C:/Users/user/Downloads/final/5_Refined Dataset_(2).csv', delimiter=",", skiprows=1)

# 데이터를 NumPy 배열로 변환 
test_array = test_data.values

In [19]:
# 모델을 사용하여 예측 수행
test_predictions = model.predict(test_array)

# 예측 결과를 이진 레이블로 변환
test_pred_labels = [1 if p > 0.5 else 0 for p in test_predictions]

# 예측 결과 출력
for i, prediction in enumerate(test_pred_labels):
    print(f"Data {i+1} relationship 성공 여부 예측: {prediction}")

20/20 [==============================] - 0s 2ms/step
Data 1 relationship 성공 여부 예측: 0
Data 2 relationship 성공 여부 예측: 0
Data 3 relationship 성공 여부 예측: 1
Data 4 relationship 성공 여부 예측: 0
Data 5 relationship 성공 여부 예측: 0
Data 6 relationship 성공 여부 예측: 0
Data 7 relationship 성공 여부 예측: 0
Data 8 relationship 성공 여부 예측: 0
Data 9 relationship 성공 여부 예측: 0
Data 10 relationship 성공 여부 예측: 0
Data 11 relationship 성공 여부 예측: 0
Data 12 relationship 성공 여부 예측: 0
Data 13 relationship 성공 여부 예측: 0
Data 14 relationship 성공 여부 예측: 0
Data 15 relationship 성공 여부 예측: 0
Data 16 relationship 성공 여부 예측: 0
Data 17 relationship 성공 여부 예측: 0
Data 18 relationship 성공 여부 예측: 0
Data 19 relationship 성공 여부 예측: 0
Data 20 relationship 성공 여부 예측: 1
Data 21 relationship 성공 여부 예측: 0
Data 22 relationship 성공 여부 예측: 0
Data 23 relationship 성공 여부 예측: 0
Data 24 relationship 성공 여부 예측: 0
Data 25 relationship 성공 여부 예측: 0
Data 26 relationship 성공 여부 예측: 0
Data 27 relationship 성공 여부 예측: 0
Data 28 relationship 성공 여부 예측: 0
Data 29 relationship 성공 여부 예측: 0

<span style="color: blue; font-size: 18pt">
8. 딥러닝 모델 저장하기
</span>

In [20]:
model.save('C:/Users/user/Downloads/final/DL_RESULT/tinder_Sequential.h5') 